In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!git clone https://github.com/clinicalml/theanomodels.git

Cloning into 'theanomodels'...
remote: Enumerating objects: 547, done.
remote: Total 547 (delta 0), reused 0 (delta 0), pack-reused 547
Receiving objects: 100% (547/547), 643.87 KiB | 1.61 MiB/s, done.
Resolving deltas: 100% (324/324), done.


In [0]:
!git clone https://github.com/clinicalml/dmm.git

Cloning into 'dmm'...
remote: Enumerating objects: 276, done.
remote: Total 276 (delta 0), reused 0 (delta 0), pack-reused 276
Receiving objects: 100% (276/276), 10.07 MiB | 10.62 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [0]:
ls

dmm/  theanomodels/


In [0]:
import sys
sys.path.append('theanomodels')
sys.path.append('dmm')

In [0]:
import os
import numpy as np
from utils.misc import loadHDF5, createIfAbsent
import glob, os, sys, time
sys.path.append('../')
from utils.misc import getConfigFile, readPickle, displayTime, downloadData, loadHDF5, getLowestError
start_time = time.time()
from   model_th.dmm import DMM
import model_th.learning as DMM_learn
import model_th.evaluate as DMM_evaluate
displayTime('importing DMM',start_time, time.time())

		< importing DMM > took  2.0026910305   seconds


In [0]:
if not os.path.exists('./midi/'):
    download_dir = './'
    files        = {}
    files['midi.zip'] = 'http://www.iro.umontreal.ca/~lisa/deep/midi.zip'
    downloadData(download_dir,files)
    os.system('unzip midi.zip;rm -rf midi.zip')
else:
    print './midi found'
    
from midi.utils import midiread, midiwrite

Execute:  wget http://www.iro.umontreal.ca/~lisa/deep/midi.zip -O .//midi.zip
Downloaded:  midi.zip 



## Train the model

In [0]:
#!THEANO_FLAGS="scan.allow_gc=False,compiledir_format=gpu3,device=cuda*" python2.7 /content/dmm/expt/train.py -infm R -ar 200 -dset tschai -uid DKF-ar
#gpuarray.preallocate=0.6,

!THEANO_FLAGS="gpuarray.preallocate=0.6,scan.allow_gc=False,compiledir_format=gpu1" python2.7 /content/dmm/expt/train.py -infm R -ar 2000 -dset jsb -uid DKF-ar


Loading linear matrices
Loading linear matrices
Found:  theanomodels/datasets/polyphonic//jsb.h5
Options:     {
dataset : jsb 
epochs : 2000 
seed : 1 
init_weight : 0.1 
reg_spec : _ 
expt_name : DKF-ar 
reg_value : 0.05 
reloadFile : ./NOSUCHFILE 
dim_stochastic : 100 
transition_layers : 2 
lr : 0.0008 
reg_type : l2 
init_scheme : uniform 
optimizer : adam 
data_type : binary 
use_generative_prior : approx 
maxout_stride : 4 
batch_size : 20 
savedir : ./chkpt-jsb 
forget_bias : -5.0 
inference_model : R 
emission_layers : 2 
savefreq : 10 
rnn_cell : lstm 
rnn_size : 600 
paramFile : ./NOSUCHFILE 
nonlinearity : relu 
dim_observations : 88 
rnn_dropout : 0.1 
dim_hidden : 200 
anneal_rate : 2000.0 
debug : False 
validate_only : False 
transition_type : mlp 
unique_id : DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar 
leaky_param : 0.0 

}
		< import DMM > took  1.50102305412   seconds
Training model from scrat

## Load trained model

In [0]:
#change the dataset to one of ['jsb','nottingham','musedata','piano']
DATASET= 'jsb'
DIR    = '/content/chkpt-'+DATASET+'/'
assert os.path.exists('/content/chkpt-'+DATASET+'/'),'Run the shell files in ../expt first'
prefix = 'DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar'
stats  = loadHDF5(os.path.join(DIR,prefix+'-EP670-stats.h5'))
epochMin, valMin, idxMin = getLowestError(stats['valid_bound'])
pfile  =  os.path.join(DIR,prefix+'-config.pkl')

params = readPickle(pfile, quiet=True)[0]
print 'Hyperparameters in: ',pfile, 'Found: ',os.path.exists(pfile)
EP     = '-EP'+str(int(epochMin))
reloadFile  =  os.path.join(DIR,prefix+EP+'-params.npz')
print 'Model parameters in: ',reloadFile
#Don't load the training functions for the model since its time consuming
params['validate_only'] = True
dmm_reloaded  = DMM(params, paramFile = pfile, reloadFile = reloadFile)

Hyperparameters in:  /content/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-config.pkl Found:  True
Model parameters in:  /content/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP670-params.npz
	<<Reloading Model>>
	<<Loading structure (/content/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-config.pkl) and model (/content/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP670-params.npz) / opt (/content/chkpt-jsb/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP670-optParams.npz) weights>>
Adding/Modifying loaded parameters:  validate_only  to  True
	<<Nparameters: 3295291>>


theanomodels/models/__init__.py:201: UserWarning: lr found in npWeights. No action taken
  warnings.warn(name+" found in npWeights. No action taken")
theanomodels/models/__init__.py:201: UserWarning: anneal found in npWeights. No action taken
  warnings.warn(name+" found in npWeights. No action taken")
theanomodels/models/__init__.py:201: UserWarning: update_ctr found in npWeights. No action taken
  warnings.warn(name+" found in npWeights. No action taken")


	<<Anneal = 1 in 2000.0 param. updates>>
	<<Building with RNN dropout:0.0>>
	<<In _LSTM_RNN_layer with dropout 0.0000>>


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


	<<Completed DMM setup>>
	<<_buildModel took : 164.4357 seconds>>


## Generate new music

In [0]:
params, zvec = DMM_evaluate.sample(dmm_reloaded, T= 100)
bin_prob     = params[0]
print 'Samples: ',bin_prob

Samples:  [[[1.69347324e-07 1.32550736e-07 1.75639626e-07 ... 1.83573463e-07
   1.66769628e-07 1.47500461e-07]
  [3.62209317e-06 3.24872662e-06 3.66665991e-06 ... 3.67286167e-06
   3.33056464e-06 3.72161082e-06]
  [1.10751134e-05 9.59530690e-06 1.06060676e-05 ... 1.16959238e-05
   1.08128055e-05 1.00529357e-05]
  ...
  [2.43004024e-08 2.66494395e-08 3.59144458e-08 ... 2.13999095e-08
   2.74396711e-08 2.95310134e-08]
  [5.24077657e-08 5.59748079e-08 6.20981372e-08 ... 5.61619548e-08
   6.70536627e-08 6.14170936e-08]
  [2.80817870e-07 2.82143996e-07 3.65105430e-07 ... 2.73596227e-07
   3.24070249e-07 3.34529282e-07]]

 [[6.72397625e-05 6.32542237e-05 6.27007360e-05 ... 7.68084047e-05
   6.26995943e-05 5.79317993e-05]
  [5.56467102e-07 3.69883128e-07 4.42860232e-07 ... 5.63053956e-07
   3.86266234e-07 3.90937275e-07]
  [2.44797522e-07 1.50958394e-07 2.02238014e-07 ... 2.23160775e-07
   2.18822071e-07 1.86588710e-07]
  ...
  [1.28452033e-08 1.16876343e-08 1.58707068e-08 ... 2.00851202e-08


In [0]:

#Parameters for music sampling taken from http://deeplearning.net/tutorial/rnnrbm.html
rval = (21, 109)
dt   = 0.3
SAVEDIR = './samples/'
createIfAbsent(SAVEDIR)
print 'Saving wav...'
for idx, sample in enumerate(bin_prob):
    piano_roll = (sample>0.5)*1.
    if idx ==4:
      print(piano_roll)
    filename = SAVEDIR+DATASET+'-'+str(idx)+'.mid'
    midiwrite(filename, piano_roll, rval, dt)
    print idx,

# print 'Converting...'
# print os.system('cd '+SAVEDIR+';timidity -Ow1S '+DATASET+'*.mid;cd ../')
# print '\nFiles\n',', '.join(os.listdir(SAVEDIR))

Saving wav...
0 1 2 3 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99


In [0]:
!tar -czvf chkpt-piano.tar.gz /content/chkpt-piano

tar: Removing leading `/' from member names
/content/chkpt-piano/
/content/chkpt-piano/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP20-optParams.npz
/content/chkpt-piano/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP40-params.npz
/content/chkpt-piano/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP10-stats.h5
/content/chkpt-piano/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP40-stats.h5
/content/chkpt-piano/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP20-params.npz
/content/chkpt-piano/DMM_lr-0_0008-dh-200-ds-100-nl-relu-bs-20-ep-2000-rs-600-rd-0_1-infm-R-tl-2-el-2-ar-2000_0-use_p-approx-rc-lstm-DKF-ar-EP0-stats.h5
/c

In [0]:
!wget https://gofile.io/?c=sLq5DE

--2019-12-04 16:05:41--  https://gofile.io/?c=sLq5DE
Resolving gofile.io (gofile.io)... 213.32.73.239, 217.182.142.131, 217.182.142.52
Connecting to gofile.io (gofile.io)|213.32.73.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23433 (23K) [text/html]
Saving to: ‘index.html?c=sLq5DE’

index.html?c=sLq5DE 100%[===================>]  22.88K  --.-KB/s    in 0.01s   

2019-12-04 16:05:41 (2.24 MB/s) - ‘index.html?c=sLq5DE’ saved [23433/23433]



In [0]:
import os,cPickle
os.system('wget '+'https://gofile.io/?c=sLq5DE -O '+os.path.join('musedata.pkl'))
os.system('wget '+'http://www-etud.iro.umontreal.ca/~boulanni/JSB%20Chorales.pickle -O '+os.path.join('jsb.pkl'))

jsbdata = cPickle.load(file('/content/jsb.pkl'))